In [ ]:
!pip install rouge_score
!pip install opendatasets

! pip install torch==2.2.0
! pip install accelerate==0.27.2

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=8d491553ea0c3e869521552636d2a9b38801bbba9167f13b4cc28503ed8fa6e9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manyli

In [ ]:
import json
import pandas as pd
import random
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import opendatasets as od
from rouge_score import rouge_scorer

In [ ]:
# od.download("https://www.kaggle.com/datasets/farneetsingh/dialogue-chat")
# od.download("https://www.kaggle.com/datasets/akch2914/samsum-dataset-for-chat-summarization")
# od.download("https://www.kaggle.com/datasets/farneetsingh/chat-conversations-with-summary")


### Dialogue dataset

In [ ]:
valid_path = "drive/MyDrive/corrector-datasets/dialogue-chat/dialogsum.dev.jsonl"
test_path = "drive/MyDrive/corrector-datasets/dialogue-chat/dialogsum.test.jsonl"
train_path = "drive/MyDrive/corrector-datasets/dialogue-chat/dialogsum.train.jsonl"

In [ ]:
def convert_json_to_df(file_path):

    data_in_json = []

    with open(file_path,"r") as file:
        for line in file:
          data_in_json.append(json.loads(line))

    df = pd.DataFrame(data_in_json)

    return df


### "#Person 1#" or "#Person 2#" is suppose to be a name. so we can create lists of random names and replace them with name

In [ ]:
names = [
    "Alice", "Bob", "Charlie", "David", "Eve", "Faythe", "Grace", "Heidi",
    "Ivan", "Judy", "Mallory", "Niaj", "Olivia", "Peggy", "Sybil", "Trent",
    "Victor", "Walter", "Xavier", "Yvonne", "Zara", "Liam", "Emma", "Noah",
    "Olivia", "Ava", "Isabella", "Sophia", "Mia", "Charlotte", "Amelia",
    "Harper", "Evelyn", "Abigail", "Emily", "Ella", "Elizabeth", "Camila",
    "Luna", "Sofia", "Avery", "Mila", "Aria", "Scarlett", "Penelope", "Layla",
    "Chloe", "Victoria", "Madison", "Eleanor", "Grace", "Nora", "Riley",
    "Zoey", "Hannah", "Hazel", "Lily", "Ellie", "Violet", "Lillian", "Zoe",
    "Stella", "Aurora", "Natalie", "Emilia", "Everly", "Leah", "Aubrey",
    "Willow", "Addison", "Lucy", "Audrey", "Bella", "Nova", "Brooklyn",
    "Paisley", "Savannah"
]

In [ ]:
def replace_name(x:pd.DataFrame):
  c = random.sample(names,2)
  x["dialogue"] = x["dialogue"].replace("#Person1#",c[0])
  x["dialogue"] = x["dialogue"].replace("#Person2#",c[1])
  x["summary"] = x["summary"].replace("#Person1#",c[0])
  x["summary"] = x["summary"].replace("#Person2#",c[1])
  return x

def replace_names_in_df(df:pd.DataFrame):
   df = df.apply(replace_name, axis=1)
   return df


In [ ]:
def convert_and_replace_names_in_df(file_path):
  df = convert_json_to_df(file_path)
  df = replace_names_in_df(df)
  return df[["dialogue","summary"]]

test_data

In [ ]:
convert_json_to_df(test_path)

,fname,dialogue,summary1,topic1,summary2,topic2,summary3,topic3
0,test_0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method,In order to prevent employees from wasting tim...,company policy,Ms. Dawson takes a dictation for #Person1# abo...,dictation
1,test_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation,#Person2# decides to follow #Person1#'s sugges...,transportation,#Person2# complains to #Person1# about the tra...,discuss transportation
2,test_2,"#Person1#: Kate, you never believe what's happ...",#Person1# tells Kate that Masha and Hero get d...,divorce,#Person1# tells Kate that Masha and Hero are g...,divorce,#Person1# and Kate talk about the divorce betw...,discuss divorce
3,test_3,"#Person1#: Happy Birthday, this is for you, Br...",#Person1# and Brian are at the birthday party ...,birthday party,#Person1# attends Brian's birthday party. Bria...,birthday party,#Person1# has a dance with Brian at Brian's bi...,birthday party
4,test_4,#Person1#: This Olympic park is so big!\n#Pers...,#Person1# is surprised at the Olympic Stadium'...,Olympic Stadium,#Person2# shows #Person1# around the construct...,sports stadium,#Person2# introduces the Olympic Stadium's fin...,Olympic Stadium
...,...,...,...,...,...,...,...,...
495,test_495,"#Person1#: Hey, Charlie, do you want to come t...",Jack invites Charlie to play a new video game ...,video game invitation,Jack asks Charlie to come over and play the ne...,\n\nentertainment activity schedule,Jack invites Charlie to play video games after...,play game
496,test_496,#Person1#: How did you get interested in count...,#Person2# explains to #Person1# about how #Per...,conversation about interest,#Person2# shares #Person2#'s career in the pas...,work experience,#Person2# tells #Person1# about #Person2#'s ow...,country music
497,test_497,"#Person1#: Excuse me, Alice, I've never used t...",Alice guides #Person1# to use the washing mach...,campus conversation,#Person1# asks Alice how to use the washing ma...,campus life conversation,#Person1# doesn't know how to use the washing ...,clothes washing
498,test_498,#Person1#: Matthew? Hi!\n#Person2#: Steve! Hav...,Steve is looking for a new place to live and M...,house renting,Matthew and Steve meet after a long time. Stev...,finding a house,Steve has been looking for a place to live. Ma...,find a house


In [ ]:
test_df = convert_json_to_df(test_path)
train_df = convert_and_replace_names_in_df(train_path)
valid_df = convert_and_replace_names_in_df(valid_path)
test_df.head()

,fname,dialogue,summary1,topic1,summary2,topic2,summary3,topic3
0,test_0,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method,In order to prevent employees from wasting tim...,company policy,Ms. Dawson takes a dictation for #Person1# abo...,dictation
1,test_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation,#Person2# decides to follow #Person1#'s sugges...,transportation,#Person2# complains to #Person1# about the tra...,discuss transportation
2,test_2,"#Person1#: Kate, you never believe what's happ...",#Person1# tells Kate that Masha and Hero get d...,divorce,#Person1# tells Kate that Masha and Hero are g...,divorce,#Person1# and Kate talk about the divorce betw...,discuss divorce
3,test_3,"#Person1#: Happy Birthday, this is for you, Br...",#Person1# and Brian are at the birthday party ...,birthday party,#Person1# attends Brian's birthday party. Bria...,birthday party,#Person1# has a dance with Brian at Brian's bi...,birthday party
4,test_4,#Person1#: This Olympic park is so big!\n#Pers...,#Person1# is surprised at the Olympic Stadium'...,Olympic Stadium,#Person2# shows #Person1# around the construct...,sports stadium,#Person2# introduces the Olympic Stadium's fin...,Olympic Stadium


### Samsun

In [ ]:
def cleaning(df):
    # Count and print the number of duplicate rows in the DataFrame
    dup = df.duplicated().sum()
    print('Number of duplicates: ', dup)

    # If there are duplicates, remove them and print confirmation
    if dup > 0:
        df.drop_duplicates(inplace=True)
        print('Duplicate values removed.')

    # Remove the 'id' column from the DataFrame
    df.drop('id', axis=1, inplace=True)

    # Calculate and print the number of missing values in each column
    missing = df.isna().sum()
    print('Number of missing values: ', missing)

    # Remove rows with missing values from the DataFrame
    df.dropna(inplace=True)

    # Return the cleaned DataFrame
    return df

In [ ]:
train_df2 = pd.read_json('./drive/MyDrive/corrector-datasets/samsum-dataset-for-chat-summarization/train.json')
valid_df2 = pd.read_json('./drive/MyDrive/corrector-datasets/samsum-dataset-for-chat-summarization/val.json')
test_df2 = pd.read_json('./drive/MyDrive/corrector-datasets/samsum-dataset-for-chat-summarization/test.json')

In [ ]:
cleaning(train_df2)

Number of duplicates:  0
Number of missing values:  summary     0
dialogue    0
dtype: int64


,summary,dialogue
0,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...
...,...,...
14727,Romeo is trying to get Greta to add him to her...,Romeo: You are on my ‘People you may know’ lis...
14728,Theresa is at work. She gets free food and fre...,Theresa: <file_photo>\r\nTheresa: <file_photo>...
14729,Japan is going to hunt whales again. Island an...,John: Every day some bad news. Japan will hunt...
14730,Celia couldn't make it to the afternoon with t...,Jennifer: Dear Celia! How are you doing?\r\nJe...


In [ ]:
cleaning(test_df2)

Number of duplicates:  0
Number of missing values:  summary     0
dialogue    0
dtype: int64


,summary,dialogue
0,Hannah needs Betty's number but Amanda doesn't...,"Hannah: Hey, do you have Betty's number?\nAman..."
1,Eric and Rob are going to watch a stand-up on ...,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...
2,Lenny can't decide which trousers to buy. Bob ...,"Lenny: Babe, can you help me with something?\r..."
3,Emma will be home soon and she will let Will k...,"Will: hey babe, what do you want for dinner to..."
4,Jane is in Warsaw. Ollie and Jane has a party....,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju..."
...,...,...
814,Benjamin didn't come to see a basketball game ...,Alex: Were you able to attend Friday night's b...
815,The audition starts at 7.30 P.M. in Antena 3.,Jamilla: remember that the audition starts at ...
816,"Marta sent a file accidentally,","Marta: <file_gif>\r\nMarta: Sorry girls, I cli..."
817,There was a meet-and-greet with James Charles ...,Cora: Have you heard how much fuss British med...


In [ ]:

cleaning(valid_df2)


Number of duplicates:  0
Number of missing values:  summary     0
dialogue    0
dtype: int64


,summary,dialogue
0,A will go to the animal shelter tomorrow to ge...,"A: Hi Tom, are you busy tomorrow’s afternoon?\..."
1,Emma and Rob love the advent calendar. Lauren ...,Emma: I’ve just fallen in love with this adven...
2,Madison is pregnant but she doesn't want to ta...,Jackie: Madison is pregnant\r\nJackie: but she...
3,Marla found a pair of boxers under her bed.,Marla: <file_photo>\r\nMarla: look what I foun...
4,Robert wants Fred to send him the address of t...,Robert: Hey give me the address of this music ...
...,...,...
813,Carla's date for graduation is on June 4th. Di...,Carla: I've got it...\r\nDiego: what?\r\nCarla...
814,Bev is going on the school trip with her son. ...,"Gita: Hello, this is Beti's Mum Gita, I wanted..."
815,Greg cheated on Julia. He apologises to her. R...,"Julia: Greg just texted me\r\nRobert: ugh, del..."
816,Marry broke her nail and has a party tomorrow....,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n..."


In [ ]:

with open('./drive/MyDrive/corrector-datasets/chat-conversations-with-summary/dialogue.txt', 'r') as file:

    lines = file.readlines()


data = []


for line in lines:

    columns = line.strip().split('~')

    data.append(columns)


df = pd.DataFrame(data)


df = df[1:]


df.rename(columns={0 : 'dialogue', 1 : 'summary'}, inplace=True)


df.reset_index(drop=True, inplace=True)


df = df[['summary', 'dialogue']]


df

,summary,dialogue
0,Alice is checking if Bob has finished the repo...,"Alice: Hey, have you finished the report? Bob:..."
1,"John and Emma discuss a new project, and Emma ...",John: Did you hear about the new project? Emma...
2,"Mike loses his keys, but Sara suggests checkin...",Mike: I can't find my keys anywhere. Sara: Did...
3,"Liam and Sophie confirm their movie plans, wit...",Liam: Are we still on for the movie tonight? S...
4,"David and Nina discuss the weather forecast, d...",David: Have you seen the weather forecast for ...
...,...,...
904,Ravi tells Mei about his culinary experiments ...,Ravi: I've been experimenting with cooking dif...
905,Sophie discusses her project to clean up the l...,Sophie: I'm working on a project to clean up o...
906,Neil shares his interest in historical novels ...,Neil: I've been exploring historical novels re...
907,Grace mentions her new blog on sustainable liv...,Grace: I started a blog about sustainable livi...


In [ ]:
# Concatenating the DataFrames
train_df = pd.concat([train_df, train_df2, valid_df, valid_df2, df])

# Reseting the index
train_df.reset_index(drop=True, inplace=True)

# Displaying the result
train_df

,dialogue,summary
0,"Olivia: Hi, Mr. Smith. I'm Doctor Hawkins. Why...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"Abigail: Hello Mrs. Parker, how have you been?...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"Violet: Excuse me, did you see a set of keys?\...",Violet's looking for a set of keys and asks fo...
3,Ivan: Why didn't you tell me you had a girlfri...,Ivan's angry because Natalie didn't tell Ivan ...
4,"Olivia: Watsup, ladies! Y'll looking'fine toni...",Malik invites Nikki to dance. Nikki agrees if ...
...,...,...
29414,Ravi: I've been experimenting with cooking dif...,Ravi tells Mei about his culinary experiments ...
29415,Sophie: I'm working on a project to clean up o...,Sophie discusses her project to clean up the l...
29416,Neil: I've been exploring historical novels re...,Neil shares his interest in historical novels ...
29417,Grace: I started a blog about sustainable livi...,Grace mentions her new blog on sustainable liv...


In [ ]:
class SummaryDataset(Dataset):
    # Initialize the dataset with a tokenizer, data, and maximum token length
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer  # Tokenizer for encoding text
        self.data = data            # Data containing dialogues and summaries
        self.max_length = max_length # Maximum length of tokens

    # Return the number of items in the dataset
    def __len__(self):
        return len(self.data)

    # Retrieve an item from the dataset by index
    def __getitem__(self, idx):
        item = self.data.iloc[idx]  # Get the row at the specified index
        dialogue = item['dialogue'] # Extract dialogue from the row
        summary = item['summary']   # Extract summary from the row

        # Encode the dialogue as input data for the model
        source = self.tokenizer.encode_plus(
            dialogue,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        # Encode the summary as target data for the model
        target = self.tokenizer.encode_plus(
            summary,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )

        # Return a dictionary containing input_ids, attention_mask, labels, and the original summary text
        return {
            'input_ids': source['input_ids'].flatten(),
            'attention_mask': source['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten(),
            'summary': summary
        }

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration


tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')


model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
# Creating an instance of the SummaryDataset class for training data
# It uses the tokenizer to process the training data (train_df)
# for model input
train_dataset = SummaryDataset(tokenizer, train_df)

# Creating an instance of the SummaryDataset class for validation data
# It uses the same tokenizer but with a different dataset (test_df2)
# for validation purposes
valid_dataset = SummaryDataset(tokenizer, test_df2)

## Fine tuning the model

In [ ]:
from transformers import TrainingArguments

# Define training arguments for the model
training_args = TrainingArguments(
    output_dir='./drive/MyDrive/corrector-model/results',          # Directory to save model output and checkpoints
    num_train_epochs=2,              # Number of epochs to train the model
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Weight decay for regularization
    logging_dir='./logs',            # Directory to save logs
    logging_steps=10,                # Log metrics every specified number of steps
    evaluation_strategy="epoch",     # Evaluation is done at the end of each epoch
    report_to='none'                 # Disables reporting to any online services (e.g., TensorBoard, WandB)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


fine tuning the model

In [ ]:
# # Initializing the Trainer object
# trainer = Trainer(
#     model=model,             # The model to be trained (e.g., our BART model)
#     args=training_args,      # Training arguments specifying training parameters like learning rate, batch size, etc.
#     train_dataset=train_dataset,  # The dataset to be used for training the model
#     eval_dataset=valid_dataset    # The dataset to be used for evaluating the model during training
# )

# # Starting the training process
# trainer.train()

In [ ]:
#corrector-model

# trainer.save_model("./drive/MyDrive/corrector-model/fine_tuned_model")

# Save the tokenizer
# tokenizer.save_pretrained("./drive/MyDrive/corrector-model/model-tokenizer")

In [ ]:
my_model = BartForConditionalGeneration.from_pretrained("./drive/MyDrive/corrector-model/fine_tuned_model")
tokenizer = BartTokenizer.from_pretrained("./drive/MyDrive/corrector-model/model-tokenizer")



In [ ]:
train_dataset

In [ ]:
valid_df

,dialogue,summary
0,"Zoe: Hello, how are you doing today?\nEmily: I...",Emily has trouble breathing. The doctor asks E...
1,Savannah: Hey Jimmy. Let's go workout later to...,Savannah invites Jimmy to go workout and persu...
2,Penelope: I need to stop eating such unhealthy...,"Penelope plans to stop eating unhealthy foods,..."
3,Luna: Do you believe in UFOs?\nRiley: Of cours...,Riley believes in UFOs and can see them in dre...
4,Amelia: Did you go to school today?\nTrent: Of...,Amelia didn't go to school today. Trent wants ...
...,...,...
495,"Ava: Now that it's the new year, I've decided ...",Ava decides to stop smoking and come out of th...
496,"Evelyn: You married Joe, didn't you? \nLiam: J...",Evelyn thought Liam married Joe. Liam denies.
497,Lily: How can I help you mam?\nVictoria: I was...,Victoria's car makes noises. Lily thinks it ne...
498,"Savannah: Hello, Amazon's customer service. Ho...",Aria calls Amazon's customer service because o...


In [ ]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer

def compute_rouge_scores(model, tokenizer, reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

def evaluate_model_on_dataframe(model, tokenizer, dataframe):
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for index, row in dataframe.iterrows():
        text = row['dialogue']
        reference_summary = row['summary']

        # Generate summary
        input_ids = tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
        generated_ids = model.generate(input_ids)

        # Decode generated summary
        generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Compute ROUGE scores
        scores = compute_rouge_scores(model, tokenizer, reference_summary, generated_summary)

        # Store scores
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    # Calculate average scores
    avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
    avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
    avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

    return avg_rouge1, avg_rouge2, avg_rougeL






# Evaluate both models
avg_rouge1_previous, avg_rouge2_previous, avg_rougeL_previous = evaluate_model_on_dataframe(model, tokenizer, valid_df.head(20))
print("next")
avg_rouge1_fine_tuned, avg_rouge2_fine_tuned, avg_rougeL_fine_tuned = evaluate_model_on_dataframe(my_model, tokenizer, valid_df.head(20))

# Print results
print("ROUGE Scores - Previous Model:")
print(f"  ROUGE-1: {avg_rouge1_previous:.4f}")
print(f"  ROUGE-2: {avg_rouge2_previous:.4f}")
print(f"  ROUGE-L: {avg_rougeL_previous:.4f}")

print("\nROUGE Scores - Fine-Tuned Model:")
print(f"  ROUGE-1: {avg_rouge1_fine_tuned:.4f}")
print(f"  ROUGE-2: {avg_rouge2_fine_tuned:.4f}")
print(f"  ROUGE-L: {avg_rougeL_fine_tuned:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


next
ROUGE Scores - Previous Model:
  ROUGE-1: 0.2802
  ROUGE-2: 0.0986
  ROUGE-L: 0.2580

ROUGE Scores - Fine-Tuned Model:
  ROUGE-1: 0.5268
  ROUGE-2: 0.3179
  ROUGE-L: 0.4755
